In [1]:
import os
import re
import selenium
import time
from selenium import webdriver
import warnings
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

driver = webdriver.Chrome(executable_path="C:/Users/Suraj/Desktop/IIIT/Wiki/chromedriver_win32/chromedriver.exe")

C:\Users\Suraj\AppData\Local\Temp/ipykernel_18764/2118644709.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="C:/Users/Suraj/Desktop/IIIT/Wiki/chromedriver_win32/chromedriver.exe")


In [2]:
import json
wikiplant = {}
# Opening JSON file
with open('wikiplant_v3.json') as json_file:
    wikiplant = json.load(json_file)

In [3]:
key = '246191'
wiki_ind = wikiplant[key]
flower_name = wiki_ind['Flowers of India ID']
#print(flower_name)
url_string = ''
for word in flower_name.split():
    url_string = url_string + '%20' + word

url_string = url_string.strip('%20')
flower_url = 'https://www.flowersofindia.net/catalog/slides/' + url_string + '.html'

def get_flower_attributes(wikiplant,flower_url):
    driver.get(flower_url)
    time.sleep(2)
    wikiplant[key]['Flower_Family'] = driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/div[1]').text

    flower_descr = driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/div[2]')
    wikiplant[key]['Flower_descr'] = flower_descr.text
    #print(flower_descr.text)

    medicinal_use = driver.find_element_by_xpath('/html/body/table/tbody/tr[3]/td/div[3]')
    wikiplant[key]['flower_medicinal_use'] = medicinal_use.text
    #print(medicinal_use.text)

    maintext = driver.find_element_by_xpath('//*[@id="maintext"]')
    page_text = maintext.text

    for line in page_text.split('\n'):
        if line.startswith('Common name'):
            wikiplant[key]['Flower_Common_names'] = line
            #print('Common Name:', line)
    return wikiplant


In [4]:
warnings.filterwarnings("ignore") 
count = 0
for key in wikiplant.keys():
    count = count+1
    if count%100==0:
        print('Processed ', count)
    wiki_ind = wikiplant[key]
    try:
        flower_name = wiki_ind['Flowers of India ID']
        url_string = ''
        for word in flower_name.split():
            url_string = url_string + '%20' + word

        url_string = url_string.strip('%20')
        flower_url = 'https://www.flowersofindia.net/catalog/slides/' + url_string + '.html'
        #print(url)
        wikiplant = get_flower_attributes(wikiplant,flower_url)
    except:
        continue
        #print(key)


Processed  100
Processed  200
Processed  300
Processed  400
Processed  500
Processed  600
Processed  700
Processed  800
Processed  900
Processed  1000
Processed  1100
Processed  1200
Processed  1300
Processed  1400
Processed  1500
Processed  1600
Processed  1700
Processed  1800
Processed  1900
Processed  2000
Processed  2100
Processed  2200
Processed  2300
Processed  2400
Processed  2500
Processed  2600
Processed  2700
Processed  2800
Processed  2900
Processed  3000
Processed  3100
Processed  3200


In [5]:
import json

# Serialize data into file:
json.dump( wikiplant, open( "wikiplant_v4.json", 'w' ) )

In [9]:
import pandas as pd
import numpy as np

df_new = pd.DataFrame(data=wikiplant)
df_new = (df_new.T)
df_new = df_new.replace(r'^\s*$', np.nan, regex=True)
#print (df)
#df.to_excel('wikiplant.xlsx')


for col in df_new.columns:
    #print(col , ':' , df[col].isna().sum())
    if df_new[col].isna().sum() >= 3000:
        df_new.drop(col, inplace=True, axis=1)
        
df_new.to_excel('wikiplant_v4.xlsx')

In [7]:
key = '246191'
wiki_ind = wikiplant[key]
flower_name = wiki_ind['flower_medicinal_use']
flower_name

'Medicinal uses: The plant is good in chronic fevers. Roots are given in jaundice, cough, bronchitis, asthma and pertussis. Leaves are applied on swellings. The root extract is taken both by Marma and Tanchangya for the relief of stomach pain.'